Algorytm Levenberga-Marquardta
1.  Oblicz ∇f(x_i).
2.  Oblicz  ̃H(xi).
3.  Oblicz	x_test=x_i− ̃H^(−1)(x_i)∇f(xi).			(44)
4.  Jezeli f(x_test)> f(x_i), to
(a) λ→8λ(mozna tez powiekszac o inny znaczny czynnik).
(b)  Id  ́z do punktu 2.
5.  Jezeli f(x_test)< f(x_i), to
(a) λ→λ/8(mozna tez zmniejszac o inny znaczny czynnik).
(b) x_i+1=x_test.
(c) Idz do punktu 1.

1.	Wybierz maksymalną liczbę iteracji M, punkt początkowy oraz parametr zakończenia ε. Ustal k = 0 oraz λ (0) = 10^4 (duża liczba).
2.	Oblicz ∇f(x^(k) )
3.	Jeśli ||∇f(x^(k) )|| ≤ ε, Zakończ;
    Jeśli k ≥ M; Zakończ;
    W przeciwnym razie idź do kroku 4).
4.	Oblicz x^(k+1) = x^(k) − h ∇2 f(x^(k) ) + λ^(k) I i−1 ∇f(x^(k) )
5.	Jeśli f(x^(k+1)) < f(x^(k) ), idź do kroku 6); W przeciwnym przypadku idź do kroku 7).
6.	Ustal λ^(k+1) = 1/2 λ^(k) , k = k + 1 i idź do kroku 2).
7.	Ustal λ^(k+1) = 2*λ^(k) i idź do kroku 4). 


In [1]:
using Calculus, Optim, LinearAlgebra, ForwardDiff


In [2]:
function krok2(funk::Function, x1::Array, k::Number)
     Grad = ForwardDiff.gradient(funk,[x1[1]^k,x1[2]^k])
end

function krok3(Grad::Array,E::Number,k::Number,M::Number)
    norma = LinearAlgebra.norm(Grad)
        if norma <= E
            print("norma mniejsz/równa  E")
            return 1
        end
        if k >= M
            print("przekroczono limit iteracji")
            return 1
        end
    return 0
end
    
function krok4(funk::Function, x1::Array, k::Number, lam::Number)
    gradient = ForwardDiff.gradient(funk,[x1[1]^k,x1[2]^k])
    gradient2 = ForwardDiff.gradient(funk,gradient)
    
    x1dokplus1 = x1[1]^k - (gradient2[1]+lam^k)^(-1) * gradient[1]
    x2dokplus1 = x1[2]^k - (gradient2[2]+lam^k)^(-1) * gradient[2]
    
    return [x1dokplus1,x2dokplus1]
end

function krok5(funkx1k1::Number,funkx1k::Number, k::Number, lam::Number)
        if funkx1k1 < funkx1k
            #krok 6
            #lam = 1/2 * lam
            #k=k+1
            #powrot do kroku 2
            return 0
        else
            #krok 7
            #lam = 2 * lam
            #powrot do kroku 4
            return 1
        end
end

krok5 (generic function with 1 method)

In [ ]:
function Marquardt(funk::Function, x1::Array)
    #krok 1
    #global
    global M = 100
    global k=0
    global E=2
    global lam=10^4
    global tbc=0
    
    i=0
    while i<=100
        
        #x1dokplus1=krok6(funk,x1)
        
        @label k2
        
        #krok 2
        Grad = krok2(funk,x1,k)
        
        #krok 3
        tbc = krok3(Grad,E,k,M)
        if tbc == 1
            return Grad
        end
            
        @label k4
        
        #krok 4
        #x1dokplus1=krok4(funk,x1,k,lam)
        x1dokplus1 = ForwardDiff.hessian(funk,x1)
        

        #krok 5
        
        
        println(x1dokplus1)
        
        funkx1k1 = funk(x1dokplus1)
        funkx1k = funk([x1[1]^k,x1[2]^k])
        #wracamy = krok5(funkx1k1,funkx1k,k,lam)
        if funkx1k1 < funkx1k
            lam = 1/2 * lam
            k=k+1
            print("skok do 2   ")
            print(k)
            @goto k2
        else
            lam = 2 * lam
            print("skok do 4    ")
            print(lam)
            #@goto k4
        end
        #=
        =#
        
        i=i+1
        k=k+1
    end
        return "Wystapil jakis blad. funkcja nigdy nie powinna tu skonczyc"
end

In [ ]:
function losowa(x)
   return  x[1]^3+8x[2]
end

Marquardt(losowa,[121,84])

In [69]:
array=[1,2]

array2=array[:]*5

2-element Array{Int64,1}:
  5
 10

In [ ]:

function zabawa()
    global ixsik
    ixsik=ixsik+26
    print(ixsik)
end

function testy()
    global ixsik=2
    zabawa()
    return ixsik
end

testy()

In [71]:
function test123()

    i=1
    while true
        i=i+1
        print(i)
        @goto a
    end
    
@label a 
end

test123()

2

In [ ]:
function goto_test1()
    print(123)
    @goto a
    print(321)
    return false
    @label a
    print(182)
    return true
end

goto_test1()

In [18]:
function goto_test1()
    if 1<2
        print("meh")
    else
    @label b
        @goto ab
        return true
    end
    
    while true
        print(1)
        @goto b
    end
    
    @label ab
    print("hem")
    return true
end

goto_test1()

meh1hem

true

In [37]:
hessian(x -> sin(x[1]) + cos(x[2]), [0.0, 0.0])

2×2 Array{Float64,2}:
 -7.5693e-7   0.0     
  0.0        -0.999999

In [35]:
function MyHessian(funk::Function, x::Number)
    k=0
    lam=10^4
    xdokplus1 = x^k - [Calculus.gradient(Calculus.gradient(funk,x^k))+lam^k]^(-1) * Calculus.gradient(funk,x^k)
end

MyHessian (generic function with 1 method)

In [120]:
#=

BACKUP!!!

#krok 4
                #hesjan = Calculus.hessian(funk,[x1[1],x1[2]])
        #=
                gradient = Calculus.gradient(funk,[x1[1]^k,x1[2]^k])
                gradient2 = Calculus.gradient(funk,gradient)
                x1dokplus1 = x1[1]^k - (gradient2[1]+lam^k)^(-1) * gradient[1]
                x2dokplus1 = x1[2]^k - (gradient2[2]+lam^k)^(-1) * gradient[2]
        =#

        
                #x2dokplus1 = x1[2]^k - (Calculus.gradient(Calculus.gradient(funk,[x1[1]^k,x1[2]^k]))+lam^k)^(-1) * Calculus.gradient(funk,[x1[1]^k,x1[2]^k])
                test = x1[1]^k
                #ka = funk(x1^k)
                #kaplus1 = funk(x1^(k+1))



        norma = LinearAlgebra.norm(Grad)
        if norma <= E
            print("norma mniejsz/równa  E")
            return norma
        end
        if k >= M
            print("przekroczono limit iteracji")
            return norma
        end




        
        if funk(x1dokplus1) < funk([x1[1]^k,x1[2]^k])
            #krok 6
            lam = 1/2 * lam
            k=k+1
            #powrot do kroku 2
        else
            #krok 7
            lam = 2 * lam
            #powrot do kroku 4
        end






#=
function krok6(funk::Function, x1::Array)
    global k
    global E
    global M
    #krok 2
    Grad = krok2(funk,x1,k)

    #krok 3
    tbc = krok3(Grad,E,k,M)
    if tbc == 1
        return Grad
    end

    #krok 4
    x1dokplus1=krok4(funk,x1,k,lam)
    return x1dokplus1
end

function krok7()
    return 0
end
=#

LoadError: syntax: incomplete: unterminated multi-line comment #= ... =#

In [74]:
?hessian

search: hessian MyHessian check_hessian



No documentation found.

`Calculus.hessian` is a `Function`.

```
# 9 methods for generic function "hessian":
[1] hessian(f) in Calculus at /home/jrun/.julia/packages/Calculus/2qBLt/src/derivative.jl:97
[2] hessian(f, x::Number) in Calculus at /home/jrun/.julia/packages/Calculus/2qBLt/src/derivative.jl:84
[3] hessian(f, x::Array{T,1}) where T<:Number in Calculus at /home/jrun/.julia/packages/Calculus/2qBLt/src/derivative.jl:90
[4] hessian(f, g) in Calculus at /home/jrun/.julia/packages/Calculus/2qBLt/src/derivative.jl:96
[5] hessian(f, g, x::Union{Array{T,1}, T}) where T<:Number in Calculus at /home/jrun/.julia/packages/Calculus/2qBLt/src/derivative.jl:66
[6] hessian(f, g, x::Union{Array{T,1}, T}, dtype::Symbol) where T<:Number in Calculus at /home/jrun/.julia/packages/Calculus/2qBLt/src/derivative.jl:60
[7] hessian(f, x::Number, dtype::Symbol) in Calculus at /home/jrun/.julia/packages/Calculus/2qBLt/src/derivative.jl:72
[8] hessian(f, x::Array{T,1}, dtype::Symbol) where T<:Number in Calculus at /home/jrun/.julia/packages/Calculus/2qBLt/src/derivative.jl:78
[9] hessian(f, g, dtype::Symbol) in Calculus at /home/jrun/.julia/packages/Calculus/2qBLt/src/derivative.jl:95
```
